In [ ]:
import os
from scipy import special
import scipy.io as sio
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter


In [ ]:
class GraphMeasures:
    def __init__(self,threshold):
        self.threshold = threshold
        self.avg_node_connect = None
        self.adj_mat = None
        return
                
    def get_adj_threshold(self, adj_mat):
        """
        Creates the adjacency matrix based on the passed in correlation matrix,
        and initialized threshold values.
        
        Params: 
            adj_mat - n by n matrix of correlation values
            
        Returns:
            adj_mat - n by n matrix of 1 if edge exists otherwise 0
        """
        
        adj_mat[adj_mat <= self.threshold] = 0
        adj_mat[adj_mat > self.threshold] = 1
        return adj_mat
        
    def get_graphs(self, run, xp_num,fmri = False):
        """
        Generates a list of graphs of a specific run type, experiment number 
        and measurement type.
        
        Params: 
            run    - (string)  run type for each graph                      (ex: "eegNF")
            xp_num - (int)     which experiment to pull the data from       (ex: 1)
            fmri   - (boolean) True if you want fMRI graphs instead of EEG  (ex: True)
            
        Returns:
            graph_list - (list) a list of networkx graphs following the given 
                                parameters.
        """
        
        base_name = "correlation/fmri/" if fmri else "correlation/"
        graph_list = []
        user = "sub-xp"
        for i in range(1,23):
            file = base_name+user+str(xp_num)+"0"+str(i)+"\\" + run + ".mat"
            try:
                mat_contents = sio.loadmat(file)
                adj_mat = np.array(mat_contents["rho"])
                adj_mat -= np.identity(63)
                adj_mat = self.get_adj_threshold(adj_mat)

                G = nx.convert_matrix.from_numpy_matrix(adj_mat)
                graph_list.append(G) 
            except FileNotFoundError: 
                continue
        return graph_list
    
    def get_subj_graphs(self, sub, run):
        """
        Generates the EEG and fMRI graphs for a given subject and a given run type
        
        Params: 
            sub - (string) subject name                            (ex: "101")
            run - (string) which experiment to pull the data from  (ex: "eegNF")
            
        Returns:
            graph_list - (list) one networkx graph for the given subject and run from
                                EEG, and one from fMRI data
        """

        base_names = ["correlation/fmri/sub-xp", "correlation/sub-xp"]
        graph_list = []
        files = [base_name+sub+"/" + run + ".mat" for base_name in base_names]
        for file in files:
            try:
                mat_contents = sio.loadmat(file)
                adj_mat = np.array(mat_contents["rho"])
                adj_mat = self.get_adj_threshold(adj_mat)

                G = nx.convert_matrix.from_numpy_matrix(adj_mat)
                graph_list.append(G) 
            except FileNotFoundError: 
                graph_list.append("None")
        
        return graph_list

In [ ]:
class GraphPlots:
    def __init__(self):
        return 

    
    def get_avg_path(self, graph_list): 
        """
        Generates average path lengths for a given graph_list
        
        Params: 
            graph_list - (list) list of networkx graph objects
            
        Returns:
            np.mean - (float) average of average path lengths from graphs in graph_list
        """
        
        path_lens = []
        for graph in graph_list: 
            for component in (graph.subgraph(c).copy() for c in nx.connected_components(graph)):
                path_lens.append(nx.average_shortest_path_length(component))
                
        return np.mean(path_lens)
    
    
    def avg_path_len_bar(self, fmri_path_lens, eeg_path_lens, title, runs):
        """
        Creates bar graphs from the fmri_path_lens and eeg_path_lens lists
        
        Params: 
            fmri_path_lens - (list) list average path lengths for fMRI data
            eeg_path_lens  - (list) list average path lengths for EEG data
            
        Returns:
            None
        """
        
        blue_bar = fmri_path_lens
        orange_bar = eeg_path_lens
        ind = np.arange(max(len(fmri_path_lens), len(eeg_path_lens)))
        plt.figure(figsize=(10,5))
        width = 0.3       
        plt.bar(ind, fmri_path_lens, width, label='fMRI ')
        plt.bar(ind + width, eeg_path_lens, width, label='EEG')

        plt.xlabel('Graph Type')
        plt.ylabel('Average Path Length')
        plt.title('Average Path Length' + title)
        plt.xticks(ind + width / 2, runs)
        plt.legend(loc='best')
        plt.savefig("data/pictures/" + title + "_avg_path_len_bar.jpg")
        plt.show()


    def get_similarity(self,graph_list):
        """
        Generates a similarity value for every 2 graphs in graph_list and stores the
        values in a n x n matrix (where n is the length of graph_list)
        
        Similarity is calculated by finding the  difference in PageRank ordering between
        two nodes with the same name (and thus the same location) for each named node.
        
        Params: 
            graph_list - (list) list of networkx graph objects
            
        Returns:
            matrix - (matrix) contains similarity values for each set of graphs where
                              matrix[i][j] represents the similarity of graph_list[i]
                              and graph_list[j]
        """
        
        central = []
        
        for graph in graph_list:
            pr = nx.pagerank(graph)
            ranked = [[key, pr[key]] for key in pr.keys()]
            sorted_ranked = sorted(ranked, key=lambda item: item[1])

            # this makes ranked work. No idea why. Seems really strange.
            sorted_ranked = [sorted_ranked[i].append(i) for i in range(len(sorted_ranked))]
            
            central.append(ranked)
        
        matrix = np.zeros((len(central),len(central)))
        for i in range(len(central)):
            for j in range(i, len(central)):
                matrix[i,j] = sum([abs(central[i][x][2]-central[j][x][2]) for x in range(len(central[i]))])
                matrix[j,i] = matrix[i,j]
        
        return matrix
    
    def graph_deg_avg_and_std(self,graph_list, run, xp_num, fmri = False):
        """
        Generates the average and standard deviation for each named node within a run
        type, given experiement, and measure type. Plots this information inside a graph
        which is then saved.
        
        Params: 
            graph_list - (list)    list of networkx graph objects
            
            The following 3 are just used for the file name for saving the graph
            
            run        - (string)  run type for each graph                      (ex: "eegNF")
            xp_num     - (int)     which experiment to pull the data from       (ex: 1)
            fmri       - (boolean) True if you want fMRI graphs instead of EEG  (ex: True)
            
        Returns:
            None
        """
        
        measure = 'fmri' if fmri else 'eeg'
        file_prefix = "data/pictures/" + measure +"/"
        file_suffix = "var_avg_xp"+str(xp_num)+"_" + run+".png"
        
        
        graph_degree_list = []
        for graph in graph_list:
            graph_degree_list.append(list(dict(graph.degree).values()))

        avg_list = np.mean(graph_degree_list,axis = 0)
        std_list = np.std(graph_degree_list,axis = 0)
        
        
        plt.bar(np.arange(0, 63), np.array(avg_list), color='blue', align='center', alpha = 0.5, label = "Avg")
        plt.bar(np.arange(0, 63), np.array(std_list) , color='red', align='center',alpha = 0.5, label = "St.Dev.")
        plt.tight_layout()
        
        plt.title("Average and St. Dev. of XP"+str(xp_num)+"_" + run)
        plt.xlabel("Node Number")
        plt.ylabel("Measure ")
        plt.legend()
        
        try:
            plt.savefig(file_prefix+file_suffix, dpi = 1000)
        except:
            os.makedirs(file_prefix)
            plt.savefig(file_prefix+file_suffix, dpi = 1000)

        plt.show()
    


 

In [ ]:
def similarity(g, subs, runs, g_plots):
    """
    Generates the average and standard deviation for each named node within a run
    type, given experiement, and measure type. Plots this information inside a graph
    which is then saved.

    Params: 
        g - (GraphMeasures object) a GraphMeasures object
        subs - (list) a list of every subject to find the similarity of
        runs - (list) a lit of every run to find the similarity of for each subject
        g_plots - (GraphPlots object) a GraphPlots object

    Returns:
        matrix - (matrix) sub by run matrix containing similarity values between EEG
                          and fMRI data for each sub and run.
    """
    
    matrix = np.zeros((len(subs),len(runs)))
    
    for i in range(len(subs)):
        for j in range(len(runs)):
            
            graph_list = g.get_subj_graphs(subs[i],runs[j])
            
            if graph_list[0] == "None" or graph_list[1] == "None": 
                   matrix[i,j] = None
                   continue
            matrix[i,j] = g_plots.get_similarity(graph_list)[0,1]
    
    return matrix


def graph_similarity(g, subs, runs, g_plots):
    """
    Graphs the similarity histogram against the similarity of two random graphs

    Params: 
        g - (GraphMeasures object) a GraphMeasures object
        subs - (list) a list of every subject to find the similarity of
        runs - (list) a lit of every run to find the similarity of for each subject
        g_plots - (GraphPlots object) a GraphPlots object

    Returns:
        None
    """
    
    mat = similarity(g, subs, runs, g_plots)
    sim_list = []

    val = []
    for i in range(181):
        graph_list = [nx.generators.random_graphs.fast_gnp_random_graph(63,.1),nx.generators.random_graphs.fast_gnp_random_graph(63,.1)]
        val.append( g_plots.get_similarity(graph_list)[0,1])


    for row in mat:
        for item in row:
            if not np.isnan(item):
                sim_list.append(item)


    _, bins, _ = plt.hist(sim_list, 20, color='blue', alpha = .5, rwidth = .9, label = "EEG/FMRI")
    plt.hist(val, bins, color='red', alpha = .5, rwidth = .9, label = "2 Random graphs")
    plt.legend()
    plt.title("Histogram of the similarity between fMRI and EEG of the same run")
    plt.xlabel("Similarity")
    plt.ylabel("Number of Runs")
    plt.savefig("data/pictures/sim_distro.png",dpi= 1000)


In [ ]:
def edge_distribution(g, subs, runs, g_plots):
    """
    Finds the distribution of the total number of edges for each graph

    Params: 
        g  - (graphMeasures object) a graph measures object
        runs_xp1 - (list) a list of runs for experiment 1
        runs_xp2 - (list)  a list of runs for experiment 2
        g_plots - (graphPlots object) a graphPlots object

    Returns:
        num_edges - (matrix) matrix where each row is an array of:
                                [the number of edges in the graph,
                                which subject,
                                which run,
                                which measurement was used]
                    for each possible graph.
    """
    
    graph_list = []
    matrix = np.zeros((len(subs),len(runs)))


    for i in range(len(subs)):
        for j in range(len(runs)):
            graphs = g.get_subj_graphs(subs[i],runs[j])

            if graphs[0] != "None":
                graph_list.append([graphs[0], subs[i], runs[j], "eeg"])
            if graphs[1] != "None":
                graph_list.append([graphs[1], subs[i], runs[j], "fmri"])


    num_edges = []    
    for graph in graph_list:
        num_edges.append([len(graph[0].edges), graph[1], graph[2], graph[3]])
    
    return num_edges



def graph_edge_distribution(g, runs_xp1, runs_xp2, g_plots):
    """
    Finds and plots the distribution of the total number of edges for each graph

    Params: 
        g  - (graphMeasures object) a graph measures object
        runs_xp1 - (list) a list of runs for experiment 1
        runs_xp2 - (list)  a list of runs for experiment 2
        g_plots - (graphPlots object) a graphPlots object

    Returns:
        None
    """
    
    num_edges = edge_distribution(g, subs, runs, g_plots)
    
    num_edges.sort()
    edge_dist_eeg = [x[0] for x in num_edges if x[3] == 'eeg']
    edge_dist_fmri = [x[0] for x in num_edges if x[3] == 'fmri']
    
    _, bins, _ = plt.hist(edge_dist_eeg, 100, color='blue', alpha = .5, rwidth = .9, label = "EEG")
    plt.hist(edge_dist_fmri, bins, color='red', alpha = .5, rwidth = .9, label = "fMRI")
    plt.legend()
    plt.title("Histogram of the total number of edges for each graph")
    plt.xlabel("Total Edges")
    plt.ylabel("Number of Graphs")
    plt.savefig("data/pictures/edge_distro.png",dpi= 1000)

In [ ]:
def node_rank(graph_list):
    """
    Generates a list of average node ranks using PageRank for the graphs in graph_list

    Params: 
        graph_list = (list) a list of networkx graphs

    Returns:
        avg_ranks - (list) Average rank for each node in the graphs. Node i's average
                           rank is avg_ranks[i]
    """
    
    central = []
        
    for graph in graph_list:
        pr = nx.pagerank(graph)
        ranked = [[key, pr[key]] for key in pr.keys()]
        sorted_ranked = sorted(ranked, key=lambda item: item[1])

        # this makes ranked work. No idea why. Seems really strange.
        sorted_ranked = [sorted_ranked[i].append(i) for i in range(len(sorted_ranked))]

        central.append(ranked)
    
    sum_ranks = [0]*63
    for graph in central:
        for node in graph:
            sum_ranks[node[0]] += node[2]
    
    avg_ranks = [val/len(graph_list) for val in sum_ranks]
    
    return avg_ranks



def graph_avg_node_rank(g, subs, runs):
    """
    Creates and saves the graphs of average node rank using the PageRank formula. Does
    this for both the EEG and FMRI data.

    Params: 
        g - (GraphMeasures object) a GraphMeasures object
        subs - (list) a list of every subject to find the similarity of
        runs - (list) a lit of every run to find the similarity of for each subject

    Returns:
        None
    """
    
    eeg_graph_list = []
    fmri_graph_list = []
    for i in range(len(subs)):
        for j in range(len(runs)):
            graphs = g.get_subj_graphs(subs[i],runs[j])

            if graphs[0] != "None":
                eeg_graph_list.append(graphs[0])
            if graphs[1] != "None":
                fmri_graph_list.append(graphs[1]) 


    eeg_node_ranks = node_rank(eeg_graph_list)
    plt.bar(np.arange(0, 63), np.array(eeg_node_ranks), color='blue', align='center')
    plt.title("Average node PageRank Ranking EEG")
    plt.xlabel("Node Number")
    plt.ylabel("Average Ranking")
    plt.legend()
    plt.savefig("data/pictures/avg_pagerank_ranking_eeg.jpg", dpi = 1000)
    plt.show()

    fmri_node_ranks = node_rank(fmri_graph_list)
    plt.bar(np.arange(0, 63), np.array(fmri_node_ranks), color='blue', align='center')
    plt.title("Average node PageRank Ranking fMRI")
    plt.xlabel("Node Number")
    plt.ylabel("Average Ranking")
    plt.legend()
    plt.savefig("data/pictures/avg_pagerank_ranking_fmri.jpg", dpi = 1000)
    plt.show()

In [ ]:
def render_cluster(graph,com_list):
    """
    Displays the communities for a given graph, and community list

    Params: 
        graph    - (graph) a networkx graph object
        com_list - (list)  a list of communities

    Returns:
        None
    """
    
    G = nx.Graph(graph)
    colors = ["red", "blue","green", "yellow", "purple", "pink"]
    g = []
    r = []
    for i in com_list:
        gi= G.subgraph(i)
        g.append(gi)
        
    nx.draw_spring(G, node_size=1, with_labels=True, alpha = 0.5)
    for gi in g:
        nx.draw_spring(gi, node_size = 5, with_labels = True, alpha = 0.5, edge_color = colors[g.index(gi)])
    plt.show()



def comm_detection(graph_list, show_graph = False):
    """
    Finds the communities in each graph in the graph list and displays the communities
    for each graph.

    Params: 
        graph_list - (list) a list of networkx graphs
        show_graphs - (boolean) boolean for if the community graphs should be displayed

    Returns:
        com_list_list - (list) list of tuples of each graph and its related community
    """
    if graph_list == []:
        return
    
    com_list_list = []
    
    for graph in graph_list: 
        comp_= nx.algorithms.community.girvan_newman(graph)
        tup = tuple(sorted(c) for c in next(comp_))
        com_list_ = []
        for i in tup:
            if(len(i) >1):
                com_list_.append(i)
        if show_graph:
            render_cluster(graph, com_list_)
        
        com_list_list.append((graph,com_list_))
            
    return com_list_list
    

In [ ]:
def get_degree_distribution(graph_list):
    """
    Generates a list of the degree for every node in the graph_list

    Params: 
        graph_list - (list) a list of networkx graphs

    Returns:
        deg - (list) a list of the degree of every node of every graph in graph_list
    """
    
    deg = []
    for g in graph_list:
        deg += [d for n, d in g.degree()]

    return deg

def zipf():
    """
    Creates an x,y pair for the Zipf distribution

    Params: 
        None

    Returns:
        x - (list) list of integers from 1 to 63
        y - (list) list of Zipf values for the corresponding x
    """
    
    a = 1. 
    x = np.arange(1., 63.)
    y = x**(-a) / sum([n**(-a) for n in range(1,63)])
    y /= max(y)
    return x,y

def eeg_fmri_graph_lists(g, runs_xp1, runs_xp2):    
    """
    Creates and returns the full list of EEG graphs, FMRI graphs

    Params: 
        g - (graphMeasures object) a graph measures object
        runs_xp1 - (list) a list of runs for experiment 1
        runs_xp2 - (list) a list of runs for experiment 2

    Returns:
        eeg_graph_list  - (list) the complete list of EEG graphs
        fmri_graph_list - (list) the complete list of fMRI graphs
    """
    
    eeg_graph_list = []
    fmri_graph_list = []

    for run in runs_xp1: 
        eeg_graph_list += g.get_graphs(run,1)
        eeg_graph_list += g.get_graphs(run,1,fmri=True)

    for run in runs_xp2: 
        eeg_graph_list += g.get_graphs(run,2)
        fmri_graph_list += g.get_graphs(run,2,fmri=True)

    return eeg_graph_list, fmri_graph_list


def plot_degree_distribution(g, runs_xp1, runs_xp2):
    """
    Finds the degree distribution for all of the EEG graphs and all the fMRI graphs, and
    builds histograms, where the distributions are compared to Zipf values.

    Params: 
        g -        (graphMeasures object) a graph measures object
        runs_xp1 - (list)                 a list of runs for experiment 1
        runs_xp2 - (list)                 a list of runs for experiment 2

    Returns:
        None
    """
    
    eeg_graph_list, fmri_graph_list = eeg_fmri_graph_lists(g, runs_xp1, runs_xp2)
    x,y = zipf()
    
    # EEG
    degree_sequence = get_degree_distribution(eeg_graph_list)
    degreeCount = Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, color="b")
    plt.plot(x,y*max(cnt), color="r")
    plt.title("EEG Degree Histogram")
    plt.savefig("data/pictures/zipfeeg.jpg", dpi = 1000)

    # fMRI
    degree_sequence = get_degree_distribution(fmri_graph_list)
    degreeCount = Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, color="b")
    plt.plot(x,y*max(cnt),color="r")
    plt.title("fMRI Degree Histogram")

    # Display
    plt.ylabel("Count")
    plt.xlabel("Degree")
    ax.set_xticklabels(deg)
    plt.savefig("data/pictures/zipffmri.jpg", dpi = 1000)
    plt.show()


In [ ]:
def graph_deg_avg_std(g, runs_xp1, runs_xp2, g_plots):
    """
    Finds and plots the degree average and standard deviation across subjects
    for each run in experiments 1 and 2. 

    Params: 
        g        - (graphMeasures object) a graph measures object
        runs_xp1 - (list)                 a list of runs for experiment 1
        runs_xp2 - (list)                 a list of runs for experiment 2
        g_plots  - (graphPlots object)    a graphPlots object

    Returns:
        None
    """
    
    for run in runs_xp1: 
        eeg_graph_list = g.get_graphs(run, 1)
        g_plots.graph_deg_avg_and_std(eeg_graph_list, run, 1)
        
        fmri_graph_list = g.get_graphs(run, 1, fmri = True)
        g_plots.graph_deg_avg_and_std(fmri_graph_list, run, 1, fmri=True)

        
    for run in runs_xp2: 
        eeg_graph_list = g.get_graphs(run, 2)
        g_plots.graph_deg_avg_and_std(eeg_graph_list, run, 2)
        
        fmri_graph_list = g.get_graphs(run, 2, fmri = True)
        g_plots.graph_deg_avg_and_std(fmri_graph_list, run, 2, fmri=True)


In [ ]:
def graph_avg_path_len(g, runs_xp1, runs_xp2, g_plots):
    """
    Finds and graphs the average path length across subjects for experiments 1 and 2

    Params: 
        g        - (graphMeasures object) a graph measures object
        runs_xp1 - (list)                 a list of runs for experiment 1
        runs_xp2 - (list)                 a list of runs for experiment 2
        g_plots  - (graphPlots object)    a graphPlots object

    Returns:
        None
    """
    
    eeg_1_path, eeg_2_path, fmri_1_path, fmri_2_path = [],[],[],[]
    
    for run in runs_xp1: 
        eeg_graph_list = g.get_graphs(run, 1)
        eeg_1_path.append(g_plots.get_avg_path(eeg_graph_list))
        
        fmri_graph_list = g.get_graphs(run, 2, fmri = True)
        fmri_1_path.append(g_plots.get_avg_path(fmri_graph_list))

        
    for run in runs_xp2: 
        eeg_graph_list = g.get_graphs(run, 2)
        eeg_2_path.append(g_plots.get_avg_path(eeg_graph_list))
        
        fmri_graph_list = g.get_graphs(run, 2, fmri = True)
        fmri_2_path.append(g_plots.get_avg_path(fmri_graph_list))
    
    
    g_plots.avg_path_len_bar(fmri_1_path, eeg_1_path, "XP1", runs_xp1)
    
    g_plots.avg_path_len_bar(fmri_2_path, eeg_2_path,"XP2", runs_xp2)

In [ ]:
def community_detection(g, runs_xp1, runs_xp2, show_graphs = False):
    """
    Finds the communities present in the runs given in runs_xp1 and runs_xp2 for
    EEG and fMRI data

    Params: 
        g - (graphMeasures object) a graph measures object
        runs_xp1 - (list) a list of runs for experiment 1
        runs_xp2 - (list) a list of runs for experiment 2
        show_graphs - (boolean) boolean for if all community graphs should be displayed

    Returns:
        eeg_comms  - (list) list of tuples of each graph and its related community
                            for EEG data
        fmri_comms - (list) list of tuples of each graph and its related community
                            for fMRI data
    """
    
    eeg_graph_list, fmri_graph_list = eeg_fmri_graph_lists(g, runs_xp1, runs_xp2)
    eeg_comms = comm_detection(eeg_graph_list, show_graphs)
    fmri_comms = comm_detection(fmri_graph_list, show_graphs)
    
    return eeg_comms, fmri_comms

# Run the methods below to see results

In [ ]:
g = GraphMeasures(0.75)
subs = ["101","102","103","104","105","106","107","108","109","110","201","202","203","204","205","206","207","210","211","213","216","217","218","219","220","221","222"]
runs_xp1 = ["eegNF","fmriNF" ,"eegfmriNF","motorloc", "MIpost", "MIpre"]
runs_xp2 = ["1dNF_run-01","1dNF_run-02","1dNF_run-03","1dNF_run-04","MIpost","MIpre","2dNF_run-01","2dNF_run-02","2dNF_run-03"]
runs = runs_xp1 + runs_xp2
g_plots = GraphPlots()


In [ ]:
graph_deg_avg_std(g, runs_xp1, runs_xp2, g_plots)

In [ ]:
graph_avg_path_len(g, runs_xp1, runs_xp2, g_plots)

In [ ]:
plot_degree_distribution(g, runs_xp1, runs_xp2)

In [ ]:
graph_edge_distribution(g, runs_xp1, runs_xp2, g_plots)

In [ ]:
graph_avg_node_rank(g, subs, runs)

In [ ]:
graph_similarity(g, subs, runs, g_plots)

In [ ]:
eeg_comms, fmri_comms = community_detection(g, runs_xp1, runs_xp2)
render_cluster(eeg_comms[0][0],eeg_comms[0][1])